# --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..

...*......

..35..633.

......#...

617*......

.....+.58.

..592.....

......755.

...$.*....

.664.598..

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [1]:
from re import compile as re_compile, finditer as re_finditer

def parseMap(input):
    splitted_input = input.split("\n")
    parsed_dict = {
        "map_size": (len(splitted_input[0]), len(splitted_input)),
        "numbers": [],
        "symbols": []
    }
    for line_number, line in enumerate(splitted_input):
        # print(line)
        number_pattern = re_compile(r"[0-9]+")
        symbol_pattern = re_compile(r"[^0-9\.]+")
        _numbers = parsed_dict["numbers"]
        _symbols = parsed_dict["symbols"]
        for m in re_finditer(number_pattern, line):
            _span = m.span()
            _numbers.append(((line_number, _span[0], _span[1]), int(m.group(0))))
        for m in re_finditer(symbol_pattern, line):
            _span = m.span()
            _symbols.append(((line_number, _span[0]), m.group(0)))
    return parsed_dict

In [2]:
with open("Day3.txt") as f:
    input = f.read()
    sum_ids = 0
    parsed_map = parseMap(input)
    _sum = 0
    for i, number in enumerate(parsed_map["numbers"]):
        number_ = number[0]
        boundaries = ((max(0, number_[1] - 1), number_[2]), (max(0, number_[0] - 1), min(number_[0] + 1, parsed_map["map_size"][1])))
        for j, symbol in enumerate(parsed_map["symbols"]):
            if symbol[0][1] >= boundaries[0][0] and symbol[0][1] <= boundaries[0][1] and symbol[0][0] >= boundaries[1][0] and symbol[0][0] <= boundaries[1][1]:
                # print(f"number {number[1]} at position {number[0]} is next to symbol {symbol[1]} at position {symbol[0]} with boundaries {boundaries}")
                _sum += number[1]
                break
    print(_sum)

537732


# --- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..

...*......

..35..633.

......#...

617*......

.....+.58.

..592.....

......755.

...$.*....

.664.598..

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [29]:
def parseGears(input):
    splitted_input = input.split("\n")
    parsed_dict = {
        # "map_size": (len(splitted_input[0]), len(splitted_input)),
        "numbers": [],
        "gears": []
    }
    for line_number, line in enumerate(splitted_input):
        # print(line)
        number_pattern = re_compile(r"[0-9]+")
        symbol_pattern = re_compile(r"[*]+")
        _numbers = parsed_dict["numbers"]
        _symbols = parsed_dict["gears"]
        for m in re_finditer(number_pattern, line):
            _span = m.span()
            item = (((_span[0], _span[1] - 1), (line_number, line_number)), int(m.group(0)))
            _numbers.append(item)
        for m in re_finditer(symbol_pattern, line):
            _span = m.span()
            item = (((_span[0], _span[1] - 1), (line_number, line_number)), m.group(0))
            _symbols.append(item)
    return parsed_dict

def getNumbersAdjacentToGears(gears, numbers):
    adj = []
    for i, gear in enumerate(gears):
        adjacent_numbers = []
        gear_ = gear[0]
        boundaries = ((gear_[0][0] - 1, gear_[0][1] + 1), (gear_[1][0] - 1, gear_[1][1] + 1))
        for j, number in enumerate(numbers):
            number_ = number[0]
            if number_[0][0] <= boundaries[0][1] and number_[0][1] >= boundaries[0][0] and number_[1][0] <= boundaries[1][1] and number_[1][1] >= boundaries[1][0]:
                adjacent_numbers.append(j)
        adj.append((i, adjacent_numbers))
    return adj


In [30]:
with open("Day3.txt") as f:
    input = f.read()
    sum_ids = 0
    parsed_map = parseGears(input)
    _numbers = parsed_map["numbers"]
    gears = getNumbersAdjacentToGears(parsed_map["gears"], _numbers)
    _sum = 0
    for gear in gears:
        gear_ = gear[1]
        if len(gear_) == 2:
            _sum += _numbers[gear_[0]][1] * _numbers[gear_[1]][1]
    print(_sum)

84883664
